In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

cache_dir = LocalFileStore('./.cache/')

llm = ChatOpenAI()

# import pandas as pd

# df = pd.read_csv('./230001data.csv')

# df.to_csv('230001data.txt', sep="\t")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\t",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./230001data.txt")
# data = test_csv.load_and_split(text_splitter=splitter)
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "당신은 스포츠 통계 분석가 입니다. 주어진 문서를 바탕으로 질문에 답하세요. 문서에 없거나 모르는 정보를 지어내지 마세요. 주어진 문서 == {context}"),
  ("human", "{question}")
])

chain = {"context": retriver, "question": RunnablePassthrough() } | prompt | llm

chain.invoke("10번 경기의 팀과 결과를 알려줘")



AIMessage(content='10번 경기의 팀은 "뉴캐슬"과 "리즈"이고, 결과는 "0 : 0"으로 무승부입니다.')